# Querying data with Duckdb

With the httpfs extension, it is possible to directly query files over HTTP(S).

For parquet files, DuckDB can use a combination of the Parquet metadata and HTTP range requests to only download the parts of the file that are actually required by the query. 

https://duckdb.org/docs/extensions/httpfs


### Dataset

| Nome da Coluna  | Nome Original | Tipo de Dados | Descrição                                                |
|-----------------|---------------|---------------|----------------------------------------------------------|
| data            | date          | datetime64[ns]| Data do registro da coleta                               |
| aisle_name      | aisle_name    | object        | Nome do corredor na loja                                 |
| product_name    | product_name  | object        | Nome do produto                                          |
| marca           | brand         | object        | Marca do produto                                         |
| preço           | price         | float64       | Preço do produto em R$                                   |
| pacote          | package       | object        | Informações da embalagem do produto (ml/kg/pacotes/etc)  |
| nome_loja       | store_name    | object        | Nome da loja (mercado, farmácia, etc)                    |
| cidade_loja     | store_city    | object        | Cidade onde a loja está localizada                       |
| termo_pesquisa  | search_term   | object        | Termo de pesquisa usado para encontrar o produto pela API|


In [ ]:
import pandas as pd

from prefect_gcp.cloud_storage import GcsBucket
from prefect_gcp import GcpCredentials

In [ ]:
import duckdb
from dotenv import load_dotenv
import os

duckdb.sql('INSTALL httpfs')
duckdb.sql('LOAD httpfs')
duckdb.sql("SET s3_endpoint='storage.googleapis.com'")


In [ ]:

# Load environment variables from .env file
load_dotenv()

# load data directory
access = os.getenv("GCS_ACCESS_KEY")
secret = os.getenv("GCS_SECRET")

# load bucket name
bucket_name = os.getenv("GCS_BUCKET_NAME")

# You will obtain the key_id from the previous step of 
# configuring settings in the Google Console.
duckdb.sql(f"SET s3_access_key_id='{access}'") 

# You will obtain the secret_access_key from the previous step of 
# configuring settings in the Google Console.
duckdb.sql(f"SET s3_secret_access_key='{secret}'") 



### /test-data

In [ ]:
import time

start_time = time.time()
query = f"SELECT * FROM 's3://{bucket_name}/test-data/19072023*/*.parquet'"
df = duckdb.sql(query).df()

end_time = time.time()
execution_time = (end_time - start_time) / 60
print(f"The query took {execution_time:.3f} seconds to execute.")

In [50]:
query = f"SELECT * FROM 's3://{bucket_name}/test-data/search_term*.parquet'"
df_json = duckdb.sql(query).df()

In [51]:
df_json

,date,aisle_name,product_name,brand,price,package,store_name,store_city,search_term
0,20-07-2023,Higiene bucal,Kit escova com creme dental sabor bubble fruit...,Colgate,24.90,2 itens,Droga Raia,FLN,Tamarillo
1,20-07-2023,Higiene bucal,Kit escova com creme dental sabor bubble fruit...,Colgate,27.49,2 itens,Panvel,FLN,Tamarillo
2,20-07-2023,Shampoo e condicionador,Shampoo crescimento forte desde a raiz,Head & Shoulders,33.58,400ml,Pague Menos,FLN,Raiz-forte
3,20-07-2023,Shampoo e condicionador,Shampoo crescimento forte desde a raiz,Head & Shoulders,22.03,200ml,Pague Menos,FLN,Raiz-forte
4,20-07-2023,Shampoo e condicionador,Shampoo PRO-V bambu Nutre e Cresce,Pantene,29.38,400ml,Pague Menos,FLN,Raiz-forte
...,...,...,...,...,...,...,...,...,...
490,20-07-2023,Produtos de limpeza banheiro e cozinha,Limpador concentrado perfumado pinho,Coala,17.90,120ml,Cobasi,FLN,Limpador de Cozinha
491,20-07-2023,Produtos de limpeza banheiro e cozinha,Limpador Perfumado Rose Petals,Sanol,6.90,500ml,Cobasi,FLN,Limpador de Cozinha
492,20-07-2023,Produtos de limpeza banheiro e cozinha,Limpador perfumado lavanda,Sanol,6.90,500ml,Cobasi,FLN,Limpador de Cozinha
493,20-07-2023,Produtos de limpeza banheiro e cozinha,Limpador para forno,Diabo Verde,33.90,440ml,Cobasi,FLN,Limpador de Cozinha


In [8]:
query = f"SELECT * FROM 's3://{bucket_name}/test-data/*.json'"
df_json = duckdb.sql(query).df()

In [10]:
df_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42109 entries, 0 to 42108
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          42109 non-null  datetime64[ns]
 1   aisle_name    42109 non-null  object        
 2   product_name  42109 non-null  object        
 3   brand         40441 non-null  object        
 4   price         42109 non-null  float64       
 5   package       42105 non-null  object        
 6   store_name    42109 non-null  object        
 7   store_city    42109 non-null  object        
 8   search_term   42109 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 2.9+ MB


In [57]:
df['search_term'][0]

'Leite'

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 976 entries, 0 to 975
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          976 non-null    object 
 1   aisle_name    976 non-null    object 
 2   product_name  976 non-null    object 
 3   brand         968 non-null    object 
 4   price         976 non-null    float64
 5   package       976 non-null    object 
 6   store_name    976 non-null    object 
 7   store_city    976 non-null    object 
 8   search_term   976 non-null    object 
dtypes: float64(1), object(8)
memory usage: 68.8+ KB


In [19]:
df = df.drop_duplicates()
df = df.fillna(0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 752 entries, 0 to 975
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          752 non-null    object 
 1   aisle_name    752 non-null    object 
 2   product_name  752 non-null    object 
 3   brand         744 non-null    object 
 4   price         752 non-null    float64
 5   package       752 non-null    object 
 6   store_name    752 non-null    object 
 7   store_city    752 non-null    object 
 8   search_term   752 non-null    object 
dtypes: float64(1), object(8)
memory usage: 58.8+ KB


In [55]:
df.search_term[0]

'Leite'

In [64]:
query = f"SELECT * FROM 's3://{bucket_name}/bronze/*/*/*/*/*.parquet'"
df_bronze_210723 = duckdb.sql(query).df()



In [63]:
df_bronze_210723.search_term.unique()

array(['haribo', 'm&m', 'twix'], dtype=object)

In [33]:
conn = duckdb.connect(database=":memory:", read_only=False)
conn.sql("INSTALL httpfs")
conn.sql("LOAD httpfs")
conn.sql(f"SET s3_access_key_id='{access}'")
conn.sql(f"SET s3_secret_access_key='{secret}'")
conn.sql("SET s3_endpoint='storage.googleapis.com'")
conn.sql("CREATE TABLE table_test AS SELECT * FROM df")
conn.sql(f"COPY table_test TO 's3://{bucket_name}/silver-data/table_test.parquet' (FORMAT PARQUET);")

## /bronze

In [65]:
query = f"SELECT * FROM 's3://{bucket_name}/bronze/*/*/*/*/*/*.parquet'"
df_bronze = duckdb.sql(query).df()

In [66]:
df_bronze.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          250 non-null    object 
 1   aisle_name    250 non-null    object 
 2   product_name  250 non-null    object 
 3   brand         250 non-null    object 
 4   price         250 non-null    float64
 5   package       250 non-null    object 
 6   store_name    250 non-null    object 
 7   store_city    250 non-null    object 
 8   search_term   250 non-null    object 
dtypes: float64(1), object(8)
memory usage: 17.7+ KB
